In [1]:
import sys
sys.path.append('/home/yan/Data2/NOMO_Project_P1/SMPL/SMPL_python_v.1.0.0/smpl/') # smpl path
from smpl_webuser.serialization import load_model
import numpy as np
import random
import math
from scipy.optimize import minimize
from scipy.optimize import fmin
import scipy
import scipy.spatial
import pickle
import os

smpl_model = '/home/yan/Data2/NOMO_Project_P1/SMPL/SMPL_python_v.1.0.0/smpl/models/basicModel_f_lbs_10_207_0_v1.0.0.pkl'

# zeros pose and shape, initialization
poseParam = np.zeros(72)  # only the arm part
shapeParam = np.zeros(10)
deg2rad = 1.0 / 180 * math.pi
poseParam[48:51] = np.multiply(deg2rad, [0, 0, -70])
poseParam[51:54] = np.multiply(deg2rad, [0, 0, 70])
# poseParam[54:57] = np.multiply(deg2rad, [0, -20, 0])
# poseParam[57:60] = np.multiply(deg2rad, [0, 20, 0])

# SMPL body model
model = load_model(smpl_model)

In [2]:
# Objective Function Optimization process
def meanDist(bodyParam):
    model.pose[:] = bodyParam[0:72]  # pose parameters
    model.betas[:] = bodyParam[72:82]    # shape parameters
    # vertices    :  model.r
    modelV = np.array(model.r)
    modelV = modelV - np.mean(modelV, 0)
    minY = np.min(modelV, 0)[1]
    modelV = modelV - [0, minY, 0]
    
    # find the closest points for modelV in the targetV, calculate the mean distance
#     dist, indexes = scipy.spatial.cKDTree(targetV, leafsize=128).query(modelV)

    dist = np.sqrt(np.sum((targetV-modelV)**2, 1)) * 1000
    
    return np.mean(dist)

In [ ]:
dataDir = '/home/yan/Data2/3D_Body_Reconstruction/Dataset/scans/NonRigidICP_Registered_NOMO3D_Dataset2/female/'
sample_list = os.listdir(dataDir)
N_sample = len(sample_list)

In [5]:
print N_sample

2675


In [6]:
for idx in range(2500, N_sample):
# for idx in range(0, N_sample):
    
    targetObj = dataDir + sample_list[idx]

    # load target obj vertices
    with open(targetObj) as f:
        content = f.readlines()
    content = [x.strip() for x in content]
    targetV = []
    targetF = []  # start from 1
    for ii in range(0, len(content)):
        x = content[ii]
        C = x.split(" ")
        if C[0] == 'v':
            CC = C[1:4]
            v = [float(vv) for vv in CC]
            targetV.append(v)
        if C[0] == 'f':
            CC = C[1:4]
            f = [int(ff) for ff in CC]
            targetF.append(f)

    targetV = np.array(targetV)
    targetV = targetV - np.mean(targetV, 0)
    minY = np.min(targetV, 0)[1]
    targetV = targetV - [0, minY, 0]
    targetF = np.array(targetF)
    
#     # save the original mesh
#     out_obj = '/home/yan/Data2/NOMO_Project_P2/Methods/ObjectiveFunctionOptimization/' 
#               + 'Optimization_Registration/NOMO3D_Dataset1/male/male_%04d.obj' % idx
#     with open(out_obj, 'w') as fp:
#         for v in targetV: # [m]
#             fp.write( 'v %f %f %f\n' % ( v[0], v[1], v[2]) )
#         for f in targetF: # Faces are 1-based, not 0-based in obj files
#             fp.write( 'f %d %d %d\n' %  (f[0], f[1], f[2]) )
            
    # initialize the parameters
    bodyParam0 = np.zeros(82)
    bodyParam0[0:72] = poseParam
    bodyParam0[72:82] = shapeParam
#     res = minimize(meanDist, bodyParam0, method='nelder-mead', options={'xtol': 1e-8, 'maxiter': 82*500, 'disp': True})
#     res = minimize(meanDist, bodyParam0, method='Powell', options={'xtol': 1e-8, 'maxiter': 82*500, 'disp': True})
#     res = minimize(meanDist, bodyParam0, method='CG', options={'xtol': 1e-8, 'maxiter': 82*500, 'disp': True})
#     res = minimize(meanDist, bodyParam0, method='BFGS', options={'xtol': 1e-8, 'maxiter': 82*500, 'disp': True})
#     res = minimize(meanDist, bodyParam0, method='Newton-CG', options={'xtol': 1e-8, 'maxiter': 82*500, 'disp': True})
    res = minimize(meanDist, bodyParam0, method='L-BFGS-B', options={'disp': True})
    #     res = fmin(meanDist, bodyParam0, xtol=0.0001, ftol=0.0001, disp=1)
#     # save results
    out_res = '/home/yan/Data2/3D_Body_Reconstruction/Dataset/scans/Optimized_Registered_NOMO3D_Dataset2/Original_A_Posed/parameter_female/' + sample_list[idx][:-3] + 'pkl'
    with open(out_res, 'w') as f:
        pickle.dump(res.x, f)
#         pickle.dump(res, f)
        
    # restore getting back the objects
#     with open(out_res) as f:
#         x = pickle.load(f)
        
    model.pose[:] = res.x[0:72]
    model.betas[:] = res.x[72:82]
#     model.pose[:] = res[0:72]
#     model.betas[:] = res[72:82]
    
    registeredV = np.array(model.r)
    registeredV = registeredV - np.mean(registeredV, 0)
    minY = np.min(registeredV, 0)[1]
    registeredV = registeredV - [0, minY, 0]
    
    out_mesh = '/home/yan/Data2/3D_Body_Reconstruction/Dataset/scans/Optimized_Registered_NOMO3D_Dataset2/Original_A_Posed/female/' + sample_list[idx]
    with open(out_mesh, 'w') as fp:
        for v in registeredV: # [m]
            fp.write( 'v %f %f %f\n' % ( v[0], v[1], v[2]) )
        for f in model.f+1: # Faces are 1-based, not 0-based in obj files
            fp.write( 'f %d %d %d\n' %  (f[0], f[1], f[2]) )
            
    if idx % 10 == 0:
        print "the " + str(idx) + "-th object prcessed"

the 2500-th object prcessed
the 2510-th object prcessed
the 2520-th object prcessed
the 2530-th object prcessed
the 2540-th object prcessed
the 2550-th object prcessed
the 2560-th object prcessed
the 2570-th object prcessed
the 2580-th object prcessed
the 2590-th object prcessed
the 2600-th object prcessed
the 2610-th object prcessed
the 2620-th object prcessed
the 2630-th object prcessed
the 2640-th object prcessed
the 2650-th object prcessed
the 2660-th object prcessed
the 2670-th object prcessed
